In [8]:
import pandas as pd
import numpy as np
from datetime import datetime
import time
import pandas as pd
import glob
import datetime 
import math

In [ ]:
start_time=int(input("Enter the start time: "))
end_time=int(input("Enter the end time: "))
start_time=time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(start_time))
end_time=time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(end_time))

In [3]:
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    
    if to_radians:
        #lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])
        lat1=np.radians(lat1)
        lon1= np.radians(lon1)
        lat2=np.radians(lat2)
        lon2= np.radians(lon2)
        
    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))


In [4]:
def EOL(start_time,end_time):
    path = r'/Users/godwinswinton/Downloads/EOL-dump' # use your path
    all_files = glob.glob(path + "/*.csv")

    li = []

    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)

    frame = pd.concat(li, axis=0, ignore_index=True)
    frame.drop("Unnamed: 0",axis=1,inplace=True)
    frame.dropna(axis=0,inplace=True)
    frame['date_time'] = pd.to_datetime(frame['tis'],unit='s')
    frame["dist"] = haversine(frame.lat.shift(), frame.lon.shift(),frame.loc[1:, 'lat'], frame.loc[1:, 'lon'])
    frame["date"] = pd.to_datetime(frame['date_time']).dt.date
    frame=frame[(frame['date_time'] >= start_time) & (frame['date_time'] <= end_time)]
    df=frame.groupby("lic_plate_no").agg({"dist":'sum'})
    over_speed=frame.groupby("lic_plate_no").agg({"osf":'sum'})
    df["Speed_violation"]=over_speed
    df=df.rename(columns={"osf":'Distance'})
    avg_speed=frame.groupby("lic_plate_no").agg({"spd":'mean'})
    df["avg_speed"]=avg_speed
    df.reset_index(level=0,inplace=True)
    trip=pd.read_csv("/Users/godwinswinton/Downloads/Trip-Info.csv")
    trip['date_time'] = pd.to_datetime(trip['date_time'], format="%Y%m%d%H%M%S")
    trip["date"] = pd.to_datetime(trip['date_time']).dt.date
    trip=trip[(trip['date_time'] >= start_time) & (trip['date_time'] <= end_time)]
    df_trip=trip.groupby(["vehicle_number","transporter_name"]).agg({"trip_id":"count"})
    df_trip.reset_index(level=0,inplace=True)
    df_trip.reset_index(level=0,inplace=True)
    df_trip=df_trip.rename(columns={"vehicle_number":"lic_plate_no"})
    df3=pd.merge(df,df_trip, on='lic_plate_no', how='left')
    df3=df3.rename(columns={"lic_plate_no":"License_plate_number",
                            "dist":"Distance","avg_speed":"Average Speed","trip_id":"Number of Trips Completed"})
    if df3.empty:
        print("The Entered value is out of range ")
    
    else:
        print(df3)
    return df3

In [5]:
df_result=EOL(start_time,end_time)

/var/folders/49/3d6v4z396fs0bgq5h32_j0rr0000gn/T/ipykernel_4732/3972672665.py:8: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, index_col=None, header=0)


    lic_plate_no       dist  Speed_violation  avg_speed
0     BR01GF5796   0.073504                0   0.000000
1     BR01GG3237   0.043182                0   0.000000
2     BR09GA5531   0.156416                0   0.000000
3     BR24GA3738   0.326815                0   0.000000
4       BR9E3138   0.144561                0   0.000000
..           ...        ...              ...        ...
230    HR61F2126  61.549147                0  27.657534
231    HR63C5168   0.001546                0   0.035714
232    HR63D6875   0.342006                0   0.000000
233    HR64A7346  69.391576                0  34.957082
234    HR65A5170  24.354643                0  22.288000

[235 rows x 4 columns]


,License_plate_number,Distance,Speed_violation,Average Speed,transporter_name,Number of Trips Completed
0,BR01GF5796,0.073504,0,0.000000,Jay Mata Di Service,32.0
1,BR01GG3237,0.043182,0,0.000000,MAA JAGDAMBA PETROLEUM SERVICES,47.0
2,BR09GA5531,0.156416,0,0.000000,Sultania Carriers Private Limited,49.0
3,BR24GA3738,0.326815,0,0.000000,Shahabad Fuel Station,25.0
4,BR9E3138,0.144561,0,0.000000,OD MOTOR SERVICE,46.0
...,...,...,...,...,...,...
232,HR61F2126,61.549147,0,27.657534,Mannat Roadlines,87.0
233,HR63C5168,0.001546,0,0.035714,S K Petro Centre,62.0
234,HR63D6875,0.342006,0,0.000000,Anju Filling Station,28.0
235,HR64A7346,69.391576,0,34.957082,HARI FILLING STATION,6.0


In [7]:
df3

NameError: name 'df3' is not defined